## Metadata VDB Search
We will use Milvus and Towhee to help searches. Towhee is used to extract the semantics of the text and return the text embedding. The Milvus vector database can store and search vectors, and return related dataset's metadata. So we first need to install [Milvus](https://github.com/milvus-io/milvus) and [Towhee](https://github.com/towhee-io/towhee).

Before getting started, please make sure that you have started a [Milvus service](https://milvus.io/docs/install_standalone-docker.md). This notebook uses [milvus 2.2.10](https://milvus.io/docs/v2.2.x/install_standalone-docker.md) and [pymilvus 2.2.11](https://milvus.io/docs/release_notes.md#2210).

<br>
<br>

`Things to look into:`
1. Are there better encoding models for Norwegian text than 'facebook/dpr-ctx_encoder-single-nq-base'?
2. What alternatives are there to RAG for search?
3. What working alternatives have already been made by others (don't reinvent the wheel)?

### Package installations

In [10]:
#! pip install --upgrade pip
#! pip3 install -q towhee pymilvus==2.2.11
#! pip3 uninstall pymilvus -y

! pip3 install -q towhee pymilvus==2.1.1
! pip install transformers -q
! pip install pandas -q
! pip install tqdm -q
! pip3 show pymilvus | grep -Ei 'Name:|Version:'
! pip3 show towhee | grep -Ei 'Name:|Version:'
! pip3 show transformers | grep -Ei 'Name:|Version:'

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Name: pymilvus
Version: 2.1.1


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Name: towhee
Version: 1.1.3


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Name: transformers
Version: 4.37.2


## 1.1 Adding embeddings for columns

The dataset is from the [Kartverket dataset metadata](https://cdn.discordapp.com/attachments/1204433663035449384/1206537816654356480/metadata_no_format.csv?ex=65dc5ee7&is=65c9e9e7&hm=3b9a88db41103ef5393294c5eaeebb60ee2229f43724cc014d4cffc92de1f384&), which contains metadata about each dataset.

The strings in the columns need to be converted to vector representations (embedding) using Towhee [text_embedding.dpr operator](https://towhee.io/text-embedding/dpr). Columns containing these new embedings should contain the original column name with `_vector` at the end.

## 1.2 Load and clean dataset for special characters

In [27]:
import pandas as pd

dataset_file = 'Metadata_excel.xlsx'
df = pd.read_excel(dataset_file)

# Replaces Norwegian letters with closest ASCII representation
def replace_norwegian_characters(text):
    if pd.isnull(text):
        return text

    char_map = {
        'å': 'aa', 'Å': 'Aa', 'æ': 'ae', 'Æ': 'Ae', 'ø': 'oe', 'Ø': 'Oe',
        'ä': 'ae', 'Ä': 'Ae', 'ö': 'oe', 'Ö': 'Oe', 'ü': 'ue', 'Ü': 'Ue'
    }
    for nor_char, ascii_char in char_map.items():
        text = text.replace(nor_char, ascii_char)
    return text

# Clean the 'abstract' column for weird substrings
def clean_abstract(text):
    if pd.isnull(text):
        return text
    text = text.replace('  v/ ', ' ').replace('\\n', ' ').replace('"', '').replace('\\', '')
    return text

# Apply cleaning functions columns
for col in df.columns:
    if df[col].dtype == 'object':
        df[col] = df[col].apply(replace_norwegian_characters)

        # Include cleaning of weird substrings
        if col == 'abstract':
            df[col] = df[col].apply(clean_abstract)

# Replace '###' with a comma as a multi-value separator
df = df.replace(to_replace='###', value=',', regex=False)

df.head()
cleaned_dataset_filename = 'cleaned_metadata.xlsx'
df.to_excel(cleaned_dataset_filename, index=False)

## 1.3 Load dataset and vectorise chosen column

In [28]:
import pandas as pd
from towhee import pipe, ops, DataCollection
from transformers import AutoTokenizer
from tqdm import tqdm

# Function to compute embeddings for a single text
def compute_embeddings(text):
    MAX_TOKENS = 510 
    inputs = tokenizer(text, return_tensors="pt", max_length=MAX_TOKENS, truncation=True)
    truncated_text = tokenizer.decode(inputs["input_ids"][0])
    return DataCollection(embeddings_pipe(truncated_text)).to_list()[0]['vec']


# Loads dataset into dataframe and recasts columns into correct datatypes
df_kartverket = pd.read_excel(cleaned_dataset_filename)
recast_to_string = ['datasetcreationdate', 'metadatacreationdate']
df_kartverket[recast_to_string] = df_kartverket[recast_to_string].astype('object')

# Fill NaN values with an empty string
df_kartverket.fillna('', inplace=True)

# Pipe converting text to embeddings (vectors)
facebook_model_name = 'facebook/dpr-ctx_encoder-single-nq-base'
mbert_model_name = 'bert-base-multilingual-uncased'
tokenizer = AutoTokenizer.from_pretrained(facebook_model_name)
chosen_model = facebook_model_name
embeddings_pipe = (
    pipe.input('text')
        .map('text', 'vec', ops.text_embedding.dpr(model_name=chosen_model))
        .output('vec')
)

# Process each column and create new columns for embeddings
column_to_vectorise = 'abstract'
tqdm.pandas(desc="Converting to vector embeddings")
df_kartverket[column_to_vectorise + '_vector'] = df_kartverket[column_to_vectorise].progress_apply(compute_embeddings)

Converting to vector embeddings: 100%|██████████| 189/189 [00:21<00:00,  8.95it/s]


## 2. Creation of Milvus collection for metadata
Before creating a collection. The database should be examined for existing collections, to avoid forever hanging collections.

In [29]:
from pymilvus import list_collections, drop_collection, connections, MilvusException

server_host = 'ebjerk.no'
server_port = '19530'

connections.connect(host=server_host, port=server_port)
print(f'Current collections in database: {list_collections()}')

collections_to_drop = ['search_article_in_medium', 'your_collection_name', 'search_article_in_medium_salesforce']

for collection in collections_to_drop:
    if len(list_collections()) == 0:
        print(f'Database has no collections to remove.')
        break
    try:
        drop_collection(collection)
    except MilvusException:
        print(f'Could not drop collection: {collection}')

2024-02-20 11:11:38,771 - 8067764224 - decorators.py-decorators:95 - ERROR: RPC error: [drop_collection], <MilvusException: (code=1, message=DescribeCollection failed: can't find collection: search_article_in_medium)>, <Time:{'RPC start': '2024-02-20 11:11:38.737515', 'RPC error': '2024-02-20 11:11:38.771733'}>
2024-02-20 11:11:38,842 - 8067764224 - decorators.py-decorators:95 - ERROR: RPC error: [drop_collection], <MilvusException: (code=1, message=DescribeCollection failed: can't find collection: your_collection_name)>, <Time:{'RPC start': '2024-02-20 11:11:38.808016', 'RPC error': '2024-02-20 11:11:38.842096'}>


Current collections in database: ['kartverket_metadata']
Could not drop collection: search_article_in_medium
Could not drop collection: your_collection_name


2024-02-20 11:11:38,914 - 8067764224 - decorators.py-decorators:95 - ERROR: RPC error: [drop_collection], <MilvusException: (code=1, message=DescribeCollection failed: can't find collection: search_article_in_medium_salesforce)>, <Time:{'RPC start': '2024-02-20 11:11:38.876989', 'RPC error': '2024-02-20 11:11:38.914928'}>


Could not drop collection: search_article_in_medium_salesforce


## 2. Creation of Milvus collection for metadata

In [31]:
from pymilvus import connections, FieldSchema, CollectionSchema, DataType, Collection, utility


connections.connect(host=server_host, port=server_port)

def kartverket_create_milvus_collection(collection_name, vector_column, dim):
    if utility.has_collection(collection_name):
        utility.drop_collection(collection_name)

    fields = [
            FieldSchema(name='schema', dtype=DataType.VARCHAR, max_length=100),  
            FieldSchema(name='id', dtype=DataType.INT64, is_primary=True, auto_id=False),
            FieldSchema(name='uuid', dtype=DataType.VARCHAR, max_length=100), 
            FieldSchema(name='hierarchyLevel', dtype=DataType.VARCHAR, max_length=100),    
            #FieldSchema(name='hierarchyLevel_vector', dtype=DataType.FLOAT_VECTOR, dim=dim), 
            FieldSchema(name='title', dtype=DataType.VARCHAR, max_length=100),   
            #FieldSchema(name="title_vector", dtype=DataType.FLOAT_VECTOR, dim=dim),

            FieldSchema(name='datasetcreationdate', dtype=DataType.VARCHAR, max_length=500),    
            FieldSchema(name='abstract', dtype=DataType.VARCHAR, max_length=2000),   
            FieldSchema(name='abstract_vector', dtype=DataType.FLOAT_VECTOR, dim=dim),   
            FieldSchema(name='keyword', dtype=DataType.VARCHAR, max_length=2000),   
            #FieldSchema(name='keyword_vector', dtype=DataType.FLOAT_VECTOR, dim=dim),   
            FieldSchema(name='geoBox', dtype=DataType.VARCHAR, max_length=100),    
            #FieldSchema(name='geoBox_vector', dtype=DataType.FLOAT_VECTOR, dim=dim),    
            FieldSchema(name='Constraints', dtype=DataType.VARCHAR, max_length=1000),   
            #FieldSchema(name='Constraints_vector', dtype=DataType.FLOAT_VECTOR, dim=dim),   

            FieldSchema(name='SecurityConstraints', dtype=DataType.VARCHAR, max_length=500),   
            #FieldSchema(name='SecurityConstraints_vector', dtype=DataType.FLOAT_VECTOR, dim=dim),   
            FieldSchema(name='LegalConstraints', dtype=DataType.VARCHAR, max_length=2000),   
            #FieldSchema(name='LegalConstraints_vector', dtype=DataType.FLOAT_VECTOR, dim=dim),   
            FieldSchema(name='temporalExtent', dtype=DataType.VARCHAR, max_length=100),    
            ##FieldSchema(name='temporalExtent_vector', dtype=DataType.FLOAT_VECTOR, dim=dim),    
            FieldSchema(name='image', dtype=DataType.VARCHAR, max_length=1000),    
            FieldSchema(name='responsibleParty', dtype=DataType.VARCHAR, max_length=500),   
            #FieldSchema(name='responsibleParty_vector', dtype=DataType.FLOAT_VECTOR, dim=dim),   

            FieldSchema(name='link', dtype=DataType.VARCHAR, max_length=500),    
            #FieldSchema(name='metadatacreationdate', dtype=DataType.VARCHAR, max_length=500), # SUS field, encoding error?
            ##FieldSchema(name='metadatacreationdate_vector', dtype=DataType.FLOAT_VECTOR, dim=dim),    
            FieldSchema(name='productInformation', dtype=DataType.VARCHAR, max_length=1000),   
            #FieldSchema(name='productInformation_vector', dtype=DataType.FLOAT_VECTOR, dim=dim),   
            FieldSchema(name='parentId', dtype=DataType.VARCHAR, max_length=100),   
    ]
    schema = CollectionSchema(fields=fields, description='search text')
    collection_columns = [field_schema.name for field_schema in schema.fields]
    collection = Collection(name=collection_name, schema=schema)
    
    index_params = {
        'metric_type': "IP",
        'index_type': "IVF_FLAT",
        'params': {"nlist": 2048}
    }
    collection.create_index(field_name=vector_column, index_params=index_params)
    return collection, collection_columns

kartverket_collection, collection_columns = kartverket_create_milvus_collection('kartverket_metadata', column_to_vectorise + '_vector', 768)

### 2.1 Creation of dataframe subset to exclude columns with complex data types

In [32]:
# Loads the correct columns from the dataframe in the correct order for milvus
df_kartverket_slice = df_kartverket[collection_columns]
df_kartverket_slice

,schema,id,uuid,hierarchyLevel,title,datasetcreationdate,abstract,abstract_vector,keyword,geoBox,Constraints,SecurityConstraints,LegalConstraints,temporalExtent,image,responsibleParty,link,productInformation,parentId
0,iso19139,37228,7a62f16f-9aeb-4c39-bf5f-e710232fa366,software,Artsfunn,,Datasettet inneholder stedfestet informasjon a...,"[-0.03340536, 0.31729478, -0.123294964, 0.1454...",Natur###Norge###Svalbard###lav###karplanter###...,2###33###57###81,",",#########,Tilgangsrestriksjoner Andre restriksjoner: Lim...,0001-01-01now,https://editor.geonorge.no/thumbnails/7a62f16f...,Frank HansenNorsk institutt for naturforskning...,###https://ipt.nina.no/,Produktspesifikasjon###Produktark###Produktsid...,
1,iso19139,21400,79013154-92ee-4647-b160-925cbc148601,dataset,Hav og is - Iskart (shapefil),,Istjenesten ved Meteorologisk institutt utarbe...,"[-0.22233136, 0.46059728, -0.05785635, -0.0937...",Oceanographic geographical features###Inspire#...,2.00###33.00###57.00###72.00,Bruksbegrensninger Ingen begrensninger paa bru...,Sikkerhetsnivaa Ugradert: Available for genera...,Tilgangsrestriksjoner Andre restriksjoner: Lim...,0001-01-01now,https://www.geonorge.no/geonetwork/srv/nor/res...,Meteorologisk instituttistjenesten@met.no###Me...,###http://polarview.met.no/,Produktspesifikasjon###Produktark###Produktsid...,
2,iso19139,240,f0083871-0d21-44e2-945f-9de9ea94d484,dataset,Losbordingsfelt,,Bordingsfelt er angitt som et geografisk punkt...,"[-0.365273, 0.5254414, 0.031845983, -0.0814479...",Aapne data###Norge digitalt###modellbaserteVeg...,2.3987###33.2045###57.5765###71.3531,",",Sikkerhetsnivaa Ugradert: Available for genera...,Tilgangsrestriksjoner Andre restriksjoner: Lim...,0001-01-01now,https://editor.geonorge.no/thumbnails/f0083871...,Stian AamotKystverket37019700Kystveien 30Arend...,###javascript:addWMSServerLayers(\https://kyst...,https://register.geonorge.no/register/versjone...,
3,iso19139,21273,e379ef5e-8851-4305-b900-44a4587cf14c,dataset,Radnett - doseratemaalestasjoner,,Datasettet inneholder straalevernets radnettst...,"[-0.03065747, 0.20453092, 0.019765006, 0.00028...",Norge digitalt###Aapne data###modellbaserteVeg...,2###33###57.00000000000001###72,Bruksbegrensninger Ingen begrensninger paa bru...,Sikkerhetsnivaa Ugradert: Available for genera...,Tilgangsrestriksjoner Andre restriksjoner: Lim...,0001-01-01now,https://editor.geonorge.no/thumbnails/e379ef5e...,Statens straalevernnrpa@nrpa.no###Direktoratet...,###https://radnett.dsa.no/index.html,Produktspesifikasjon###Produktark###Produktsid...,
4,iso19139,37251,41ccca92-2ae9-43c9-9a45-b3d6424d1633,dataset,Predikert utbredelse og tetthetsfordeling av s...,,Basert paa gamle og nye data for forekomst av ...,"[0.11961786, 0.3978122, 0.1161223, -0.2759921,...",Species distribution###Norge digitalt###modell...,2###33###57###81,Bruksbegrensninger Ingen ###IngenNo conditions...,Sikkerhetsnivaa Ugradert: Available for genera...,Tilgangsrestriksjoner Andre restriksjoner: Lim...,0001-01-01now,https://www.geonorge.no/geonetwork/srv/nor/res...,Norsk institutt for naturforskningfrank.hansse...,###http://www.seapop.no/no/spread/open-sea/spe...,Produktspesifikasjon###Produktark###Produktsid...,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184,iso19139,55086,65509eba-5301-473c-8a91-84b2bba59173,dataset,Sentrumssoner 2016,,Inneholder avgrensning av og statistikk knytte...,"[-0.23135546, 0.57064974, 0.2820142, -0.066753...",Land use###Norge digitalt###geodataloven###fel...,2###33###57.00000000000001###72,",",Sikkerhetsnivaa Ugradert: Available for genera...,Tilgangsrestriksjoner Andre restriksjoner: Lim...,0001-01-01now,https://editor.geonorge.no/thumbnails/65509eba...,Erik EngelienStatistisk sentralbyraaerik.engel...,###https://nedlasting.geonorge.no/api/capabili...,http://www.ssb.no/natur-og-miljo/_attachment/1...,55b401f3-2ea1-4045-9f87-22ac9d6ecf66
185,iso19139,152,75461f65-eaea-4495-b2ab-3bc04d261669,dataset,Statistiske enheter grunnkretser - histori

### 2.2 Insert the subset dataframe data into Milvus collection

In [33]:
from towhee import ops, pipe, DataCollection

insert_pipe_kartverket = (pipe.input('df_kartverket_slice')
                   .flat_map('df_kartverket_slice', 'data', lambda df: df.values.tolist())
                   .map('data', 'res', ops.ann_insert.milvus_client(host=server_host, 
                                                                    port=server_port,
                                                                    collection_name='kartverket_metadata'))
                   .output('res')
)

%time _ = insert_pipe_kartverket(df_kartverket_slice)


kartverket_collection.load()
kartverket_collection.num_entities

CPU times: user 681 ms, sys: 66.4 ms, total: 747 ms
Wall time: 7.86 s


189

## 3. Prepare pipe that accepts queries

In [34]:
import numpy as np
# Variables specifying what column and collection to perform ANN comparrison against
vector_columns = [column_to_vectorise + '_vector']
collection_name = 'kartverket_metadata'

print(df_kartverket.columns)
# What columns to return for view
response_output = [
       'id', 'id', 'title', # required to have two 'id'
       'abstract', 'keyword', 'Constraints',
       'SecurityConstraints', 'LegalConstraints',
       'responsibleParty', 'productInformation', 'parentId'
]


demo_pipe = (pipe.input('query')
                    .map('query', 'vec', ops.text_embedding.dpr(model_name=chosen_model))
                    .map('vec', 'vec', lambda x: x / np.linalg.norm(x, axis=0))
                    .flat_map('vec', ('id', 'score', 'id', 'title', # required to have two 'id'
                            'abstract', 'keyword', 'Constraints',
                            'SecurityConstraints', 'LegalConstraints',
                            'responsibleParty', 'productInformation', 'parentId'), 
                                       ops.ann_search.milvus_client(host=server_host, 
                                                                    port=server_port,
                                                                    collection_name=collection_name,
                                                                    vector_field=vector_columns,
                                                                    output_fields=response_output, 
                                                                    limit=5))  
                    .output(*['query', 'score'], *response_output)
               )

Index(['schema', 'uuid', 'id', 'hierarchyLevel', 'title',
       'datasetcreationdate', 'abstract', 'keyword', 'geoBox', 'Constraints',
       'SecurityConstraints', 'LegalConstraints', 'temporalExtent', 'image',
       'responsibleParty', 'link', 'metadatacreationdate',
       'productInformation', 'parentId', 'abstract_vector'],
      dtype='object')


## 3.1 Query against metadata collection

In [35]:
kartverket_question1 = 'Roerledningssystem med tilhoerende installasjoner, utgjoer transportsystemene for petroleum fra Norsk kontinentalsokkel'
print(f'\n"{kartverket_question1}" search:')
res_kartverket1 = demo_pipe(kartverket_question1)
DataCollection(res_kartverket1).show()


"Roerledningssystem med tilhoerende installasjoner, utgjoer transportsystemene for petroleum fra Norsk kontinentalsokkel" search:


## 4. Format response into GPT API request
After the query is vectorised, and then compared against the database's vector column to find the most simmilar result(s). The most similar result's metadata can be formatted into a instruction with example for GPT. This way, we can instruct GPT how to answer the query by providing a demonstration. Before prompting with the query's current context, and the instruction to answer.

![alt text](image.png)

In [ ]:
metadata_demonstration = '''query	score	id	id	title	abstract	keyword	Constraints	SecurityConstraints	LegalConstraints	responsibleParty	productInformation	parentId
Hvor er bomberom?	118.3987045288086	22788	22788	Magasin	Database over regulerte innsjøer. Egenskapsdata er vassdragsnr., magasinnr., navn, laveste og høyeste regulerte vannstand (m.o.h...	Annet###Åpne data###Norge digitalt###modellbaserteVegprosjekter###fellesDatakatalog###Energi###Norge fastland###magasin###vannkr...	Bruksbegrensninger Ingen ###Ingen	Sikkerhetsnivå Ugradert: Available for general disclosure#########	Tilgangsrestriksjoner Andre restriksjoner: Limitation not listed######Andre restriksjonerÅpne data###Åpne data###Brukerrestriksj...	Seming Haakon SkauNorges vassdrags- og energidirektoratgisstotte@nve.no###NVE - Seksjon for geomatikk og dataanalyse/IGDNorges v...	Produktspesifikasjon###Produktark###Produktside###Tegnforklaring###dekningsoversikt###hjelp###dekningsoversikt rutenett###deknin...
Hvor er bomberom?	123.4372329711914	21492	21492	Bunnsedimenter (kornstørrelse), detaljert	Datasettet viser kornstørrelsessammensetning i sjøbunnssedimentenes øvre del (øverste 0-10 cm av sjøbunnen). I egenskapstabellen...	Sea regions###Inspire###Norge digitalt###geodataloven###Mareano###ØkologiskGrunnkart###MarineGrunnkart###fellesDatakatalog###Geo...	Bruksbegrensninger Detaljnivået på datasettet tilsier bruk innenfor kartmålestokken: 1:20.000 - 1:150.000. ###Detaljnivået på d...	Sikkerhetsnivå Ugradert: Available for general disclosure#########	Tilgangsrestriksjoner Andre restriksjoner: Limitation not listed######Andre restriksjonerÅpne data###Åpne data###Brukerrestriksj...	Aave LeplandNorges geologiske undersøkelseDataadministrator4773904000Leiv Eirikssons vei 39TrondheimAave.Lepland@ngu.nohttp://ww...	https://register.geonorge.no/produktspesifikasjoner/bunnsedimenter-kornstørrelseProduktspesifikasjon###https://register.geonorge...
Hvor er bomberom?	124.96815490722656	68030	68030	Bunnsedimenter (kornstørrelse), oversikt	Datasettet viser kornstørrelsessammensetning i sjøbunnssedimentenes øvre del. Kornstørrelsesdata er basert på analyser av sjøbun...	Sea regions###Inspire###Norge digitalt###geodataloven###Mareano###fellesDatakatalog###Geologi###Norge###Nordsjøen###Norskehavet#...	Bruksbegrensninger Detaljnivået på datasettet tilsier bruk innenfor kartmålestokken: 1:2000.000 - 1:10.000.000 ###Detaljnivået ...	Sikkerhetsnivå Ugradert: Available for general disclosure#########	Tilgangsrestriksjoner Andre restriksjoner: Limitation not listed######Andre restriksjonerÅpne data###Åpne data###Brukerrestriksj...	Aave LeplandNorges geologiske undersøkelseDataadministrator4773904000Leiv Eirikssons vei 39TrondheimAave.Lepland@ngu.nohttp://ww...	https://register.geonorge.no/produktspesifikasjoner/bunnsedimenter-kornstørrelseProduktspesifikasjon###https://register.geonorge...
Hvor er bomberom?	125.0992431640625	75832	75832	Vegetasjon - Naturtyper	Kommunvise kartlag i datasettet Vegetasjon som viser naturtyper som er viktig for biologisk mangfold og som helt eller delvis ka...	Land cover###Åpne data###Norge digitalt###fellesDatakatalog###Landbruk###Natur###Norge fastland###Vegetasjon###Arealdekke###Vege...	Bruksbegrensninger Ingen ###Ingen	Sikkerhetsnivå Ugradert: Available for general disclosure#########	Tilgangsrestriksjoner Andre restriksjoner: Limitation not listed######Andre restriksjonerÅpne data###Åpne data###Brukerrestriksj...	Ingvild NystuenNorsk institutt for bioøkonomiSeksjonsleder64 94 80 0064 94 80 01Raveien 9ÅsAkersjusPostboks 115, 1430 ÅsNorgegis...	http://www.skogoglandskap.no/seksjoner/nedlastingsinformasjonNedlastingsinformasjon###http://www.skogoglandskap.no/kart/vegetasj...
Hvor er bomberom?	125.26884460449219	69607	69607	Relativ bunnhardhet, åpne data	Relativ bunnhardhet er rasterdata som viser reflektivitetstyrke fra sjøbunnen. Reflektivitetsstyrke sier noe om sjøbunnens akust...	Geology###Åpne data###Norge digitalt###MarineGrunnkart###modellbaserteVegprosjekter###fellesDatakatalog###Geologi###Norge###Bare...	Bruksbegrensninger Ingen begrensninger på bruk er oppgitt. ###Ingen begrensninger på bruk er oppgitt.No conditions apply	Sikkerhetsnivå Ugradert: Available for general disclosure#########	Tilgangsrestriksjoner Andre restriksjoner: Limitation not listed######Andre restriksjonerÅpne data###Åpne data###Brukerrestriksj...	Aave LeplandNorges geologiske undersøkelseDataadministrator, Maringeologi, NGU4773904000Leiv Eirikssons vei 39TrondheimAave.Lepl...	Produktspesifikasjon###https://register.geonorge.no/register/versjoner/produktark/norges-geologiske-undersokelse/relativ-bunnhar...'''
metadata_link = 'https://ipt.nina.no/'

query = f'Finn dataset med magasin'
instruction_gpt = f'Skriv en respons som finner det mest korresponderende datasettet fra metadata for spørringen:'
demonstration = f'Spørring: Hvor er bomberom?\nMetadata: {metadata_demonstration}\nRespons: Ut i fra spørringen ser det ut som at du leter etter datasettet Tilfluktsrom (nedlastningslink: {metadata_link}). Dette datsettet inneholder informasjon om lokasjonene til alle offentlige tilfluktsrom (bomberom) i Norge'
current_query_instruction = f'Spørring:{query}\nMetadata: {metadata_demonstration}\nRespons:'
API_text = f'{instruction_gpt}\n{demonstration}\n\n{current_query_instruction}'
print(API_text)